In [1]:
import sys
import os
sys.path.insert(0, os.path.abspath('../src/'))
sys.path.insert(0, os.path.abspath('../src/lib/KitNET-py'))

In [22]:
import cicids2017
from pathlib import Path
from Kitsune import KitsuneAnomalyDetector


RES_PATH = "."
DATASETPATH_CACHE = Path(f"../dataset/cache")
DATASETPATH = Path("../dataset/CICIDS2017_ntop.pkl")


dataset = cicids2017.load_dataset(DATASETPATH_CACHE)
tr = dataset["TR"]

kitsune = KitsuneAnomalyDetector(1)
kitsune.fit(tr)

Feature-Mapper: train-mode, Anomaly-Detector: off-mode
The Feature-Mapper found a mapping: 19 features to 19 autoencoders.
Feature-Mapper: execute-mode, Anomaly-Detector: train-mode
Feature-Mapper: execute-mode, Anomaly-Detector: exeute-mode


In [29]:
from sklearn import metrics
import pandas as pd


dt = [dataset["DT"]]

betaset = [ 5e-2, 1e-2, 5e-3, 1e-3, 5e-4, 1e-4 ]
mset = [ 1, 2, 4, 6, 10, 14, 19 ]
dt_res = pd.DataFrame()
for par_m in mset:
    kitsune = KitsuneAnomalyDetector(par_m)
    kitsune.fit(tr)
    for par_beta in betaset:
        beta_res = { "beta": par_beta, "m": par_m }
        netdata = kitsune.isanomaly(dt, par_beta)
        Y = pd.concat(dt)["_isanomaly"]
        Y = (Y=="none").replace({True: 0, False: 1}).values
        y_hat = netdata["_y_hat"].values

        metrics_rep = [ metrics.roc_auc_score,
                        metrics.precision_score, metrics.recall_score,
                        metrics.accuracy_score, metrics.f1_score]
        for m in metrics_rep:
            mres = m(Y, y_hat)
            beta_res[m.__name__] = mres

        tn, fp, fn, tp = metrics.confusion_matrix(Y, y_hat, normalize="all").ravel()
        beta_res["tn"] = tn
        beta_res["fp"] = fp
        beta_res["fn"] = fn
        beta_res["tp"] = tp
        dt_res = dt_res.append(beta_res, ignore_index=True)

Feature-Mapper: train-mode, Anomaly-Detector: off-mode
The Feature-Mapper found a mapping: 19 features to 19 autoencoders.
Feature-Mapper: execute-mode, Anomaly-Detector: train-mode
Feature-Mapper: execute-mode, Anomaly-Detector: exeute-mode
Feature-Mapper: train-mode, Anomaly-Detector: off-mode
The Feature-Mapper found a mapping: 19 features to 14 autoencoders.
Feature-Mapper: execute-mode, Anomaly-Detector: train-mode
Feature-Mapper: execute-mode, Anomaly-Detector: exeute-mode
Feature-Mapper: train-mode, Anomaly-Detector: off-mode
The Feature-Mapper found a mapping: 19 features to 12 autoencoders.
Feature-Mapper: execute-mode, Anomaly-Detector: train-mode
Feature-Mapper: execute-mode, Anomaly-Detector: exeute-mode
Feature-Mapper: train-mode, Anomaly-Detector: off-mode
The Feature-Mapper found a mapping: 19 features to 10 autoencoders.
Feature-Mapper: execute-mode, Anomaly-Detector: train-mode
Feature-Mapper: execute-mode, Anomaly-Detector: exeute-mode
Feature-Mapper: train-mode, Anom

In [30]:
dt_res.sort_values("roc_auc_score", ascending=False)

,accuracy_score,beta,f1_score,fn,fp,m,precision_score,recall_score,roc_auc_score,tn,tp
2,0.858964,0.0050,0.408621,0.102179,0.038857,1.0,0.556338,0.322888,0.638563,0.810238,0.048725
8,0.880140,0.0050,0.406314,0.109889,0.009971,2.0,0.804435,0.271798,0.630028,0.839124,0.041016
32,0.875925,0.0050,0.344378,0.118318,0.005757,14.0,0.849866,0.215940,0.604580,0.843339,0.032586
14,0.873664,0.0050,0.303683,0.123355,0.002981,4.0,0.902357,0.182561,0.589525,0.846114,0.027549
20,0.872533,0.0050,0.293044,0.124486,0.002981,6.0,0.898601,0.175068,0.585779,0.846114,0.026419
26,0.870477,0.0050,0.274194,0.126439,0.003084,10.0,0.888060,0.162125,0.579247,0.846012,0.024465
25,0.868318,0.0100,0.252189,0.128701,0.002981,10.0,0.881633,0.147139,0.571814,0.846114,0.022204
7,0.868318,0.0100,0.251315,0.128803,0.002878,2.0,0.884774,0.146458,0.571534,0.846217,0.022101
13,0.868113,0.0100,0.249269,0.129009,0.002878,4.0,0.883817,0.145095,0.570853,0.846217,0.021896
19,0.868010,0.0100,0.248244,0.129112,0.002878,6.0,0.883333,0.144414,0.570512,0.846217,0.021793
